In [ ]:
!pip install transformers torch datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 14.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# Load the tokenizer & model
model_name = "ealvaradob/bert-finetuned-phishing"  # Replace with actual Hugging Face model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# Set device (GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/845 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 1024, padding_idx=0)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-23): 24 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm((1024,

In [ ]:
def classify_url(url):
    # Tokenize the input URL
    inputs = tokenizer(url, truncation=True, padding=True, return_tensors="pt").to(device)

    # Perform inference
    with torch.no_grad():
        outputs = model(**inputs)

    # Get prediction
    probabilities = torch.nn.functional.softmax(outputs.logits, dim=-1)
    predicted_class = torch.argmax(probabilities, dim=-1).item()

    return "Phishing" if predicted_class == 1 else "Legitimate"


In [ ]:
test_urls = [
    "https://www.google.com",  # Legitimate
    "http://secure-bank-login.com/account/verify",  # Phishing
    "https://www.paypal.com/signin",  # Legitimate
    "http://paypal.com-secure-login.xyz/signin"  # Phishing
]

for url in test_urls:
    result = classify_url(url)
    print(f"URL: {url} → Prediction: {result}")

URL: https://www.google.com → Prediction: Legitimate
URL: http://secure-bank-login.com/account/verify → Prediction: Phishing
URL: https://www.paypal.com/signin → Prediction: Phishing
URL: http://paypal.com-secure-login.xyz/signin → Prediction: Phishing


In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 10.9 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from datasets import load_dataset
from transformers import Trainer, TrainingArguments
file = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/NewURLBASEDTASK/URL dataset.csv')
file.head()
file.tail()
file['type'].value_counts(normalize=True)


# dataset = load_dataset("csv", data_files={"train": "train.csv", "test":"test.csv"})
# dataset =  dataset.map(lambda e: tokenizer(e["url"], truncation=True, padding="max_length",batched=True)



,proportion
type,
legitimate,0.768006
phishing,0.231994


In [ ]:
# Map 'phishing' to 1 and 'legitimate' to 0 in the 'status' column as target
file['type'] = file['type'].map({'phishing': 1, 'legitimate': 0})
print("Dataset Preview:")
file.head()

Dataset Preview:


,url,type
0,https://www.google.com,0
1,https://www.youtube.com,0
2,https://www.facebook.com,0
3,https://www.baidu.com,0
4,https://www.wikipedia.org,0


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Convert URLs into numerical vectors using TF-IDF
vectorizer = TfidfVectorizer(max_features=5000)  # Use top 5000 features
X = vectorizer.fit_transform(file["url"])
y = file["type"]

In [ ]:
!pip install imblearn

In [ ]:
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split

# Assume `X` contains features and `y` contains labels (0 = Legitimate, 1 = Phishing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Apply SMOTE to balance classes
smote = SMOTE(sampling_strategy='auto', random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

print("Original class distribution:", y_train.value_counts(normalize=True))
print("Balanced class distribution:", y_train_smote.value_counts(normalize=True))


Original class distribution: type
0    0.768637
1    0.231363
Name: proportion, dtype: float64
Balanced class distribution: type
0    0.5
1    0.5
Name: proportion, dtype: float64


In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from datasets import Dataset
# Convert Pandas DataFrame to Hugging Face Dataset
dataset = Dataset.from_pandas(file)

In [ ]:
from transformers import AutoTokenizer
from datasets import Dataset, DatasetDict
from sklearn.model_selection import train_test_split
import pandas as pd

# Load the tokenizer for BERT-based models
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# Convert pandas DataFrame to Hugging Face Dataset
dataset = Dataset.from_pandas(file)  # 'new' is your DataFrame

# Tokenization function
def preprocess_function(file):
    return tokenizer(file["url"], padding="max_length", truncation=True)

# Apply tokenization
tokenized_dataset = dataset.map(preprocess_function, batched=True)

# Ensure dataset only contains necessary features
tokenized_dataset = tokenized_dataset.remove_columns(["url"])  # Remove raw text column
tokenized_dataset = tokenized_dataset.rename_column("type", "labels")  # Rename target column

# Split into train and test
train_test_split = tokenized_dataset.train_test_split(test_size=0.2, seed=42)

# Convert to DatasetDict
tokenized_dataset = DatasetDict({
    "train": train_test_split["train"],
    "test": train_test_split["test"]
})

# Check the dataset structure
print(tokenized_dataset)


Map:   0%|          | 0/450176 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 360140
    })
    test: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 90036
    })
})


DatasetDict({
    train: Dataset({
        features: ['url', 'type'],
        num_rows: 360140
    })
    test: Dataset({
        features: ['url', 'type'],
        num_rows: 90036
    })
})


In [ ]:
from transformers import TrainingArguments, Trainer
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

training_args = TrainingArguments(
    output_dir="./bert_model",
    eval_strategy="epoch",  # ✅ Updated from evaluation_strategy
    save_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    weight_decay=0.01,
    fp16=torch.cuda.is_available(),  # Mixed precision training for faster performance
)

trainer = Trainer(
    model=model.to(device),  # Move model to GPU
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
)

trainer.train()


NameError: name 'model' is not defined